In [ ]:
#Bowel Movement Frequency
#Proteomics Analysis
#James Johnson
#Credit to Christian Diener, PhD as noted
#v 1-18-22

# Use this Arivale snapshot path
#frozen_ss_path='/shared-data/snapshots/arivale_snapshot_ISB_2020-03-16_2156'

# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

# Get the library
from arivale_data_interface import *
import arivale_data_interface as adi

# Set display options
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.display import display

# Apply these settings so the library knows where the data is
adi.apply_isb_config()

# If you want plots to show up in the notebook you need to run this
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

In [ ]:
#Obtain Bowel Movement Frequency (BMF) data:
bowel = adi.get_snapshot('assessments')
bowel = bowel.sort_values('days_in_program',ascending=True) # Sort df by days first
bowel = bowel.drop(columns = ['vendor','days_in_program','days_since_first_call','days_since_first_draw']) # Drop days and vendor, unneeded from this point
bowel = bowel.drop_duplicates(subset='public_client_id') # Keep only the first timepoint for each public_client_id
bowel = bowel.fillna(0) # Fill in empty BMF data with zero
bowel = bowel.set_index('public_client_id') # Arrange df by index of PCI
bowel = bowel[['assessment:digestion:bowel-movements:enum']]
bowel = bowel.rename(columns={"assessment:digestion:bowel-movements:enum":"bowel"}) # rename column to "bowel"
bowel = bowel[bowel['bowel'] != 0] #remove participants with no bowel data, reduce from 5764 -> 3955 individuals
bowel_list = ['(1) 2 or fewer times per week', '(2) 3-6 times per week', '(3) 1-3 times daily', '(4) 4+ times daily']
bowel_r = [1,2,3,4] #convert bowel responses to bowel movement frequency per week (minimum of each range, 1x, 3x, 7x, 28x)
bowel.bowel = bowel.bowel.replace(to_replace = bowel_list, value = bowel_r)
# 3955 individuals
bowel

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,bowel
public_client_id,
01092042,3
01568974,3
01684342,2
01601230,3
01124029,3
...,...
01395286,3
01232614,3
01181682,3


In [ ]:
#Now, get the Assessments data
bio=adi.get_snapshot('clients')
a = bio['sex']
b = bio['age']
c = bio['public_client_id']
biodf = pd.DataFrame({'sex':a, 'age':b, 'public_client_id':c})
biodf = biodf[['sex','age','public_client_id']]
biodf = biodf.set_index('public_client_id')
biodf = biodf.dropna()
biodf #6133 individuals

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,sex,age
public_client_id,,
01000552,M,54.0
01000978,F,52.0
01001181,F,48.0
01001298,F,61.0
01001467,F,31.0
...,...,...
QIRL,F,44.0
ZU273983,F,32.0
ZU612255,F,35.0


In [ ]:
#Get BMI data
bmi = adi.get_snapshot('weight_monthly')
bmi = bmi[['public_client_id','days_in_program','BMI_CALC']]
bmi = bmi.sort_values('days_in_program',ascending=True)
bmi = bmi.sort_values('public_client_id',ascending=True)
bmi = bmi.drop('days_in_program', axis=1)
bmi = bmi.drop_duplicates('public_client_id')
bmi = bmi.set_index('public_client_id')
bmi = bmi.dropna()
bmi #5567 individuals

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,BMI_CALC
public_client_id,
01000261,22.594123
01000552,31.619378
01001181,39.047808
01001298,25.948604
01001548,36.119222
...,...
QIRL,31.942410
ZU273983,32.042139
ZU612255,36.045023


In [ ]:
#Obtain "eGFR" calculated GFR data (credit: Alexandra Ralevski) code chunk below:
################################################################################################
chems = adi.get_snapshot('chemistries', clean=True)
cl = adi.get_snapshot('clients', clean=True)
chems_dems = join_clients(cl, chems)
#these are the relevant column names
[col for col in chems.columns if 'CREAT' in col]
[col for col in chems.columns if 'GFR' in col]
def f(sex, age, creatinine): 
    if sex == 'F':
        if (creatinine/0.7) <=1:
            return 142*(((creatinine/0.7)**-0.241) * (0.9938**age) * 1.012)
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age) * 1.012)
    if sex == 'M':
        if (creatinine/0.9) <=1:
            return 142*(((creatinine/0.9)**-0.302) * (0.9938**age))
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age))

creat_calc = chems_dems[['public_client_id','sex', 'age', 'CREATININE_ENZ__SER']]

creat_calc['eGFR'] = creat_calc.apply(lambda x: f(x.sex, x.age, x.CREATININE_ENZ__SER), axis=1)
#creat_calc.to_csv('eGFR.csv',index = False)
################################################################################################

#prepare eGFR dataframe:
eGFR = creat_calc
#eGFR = eGFR.set_index('public_client_id')
eGFR = eGFR[['public_client_id','eGFR']] #select relevant columns
#drop duplicate entries to reduce 12316 entries across all individuals
#to 6133 individuals
eGFR = eGFR.drop_duplicates('public_client_id')
eGFR = eGFR.set_index('public_client_id')
eGFR = eGFR.dropna()
eGFR

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,eGFR
public_client_id,
01001298,99.024207
01001467,100.959522
01001548,105.411124
01001621,69.439409
01001661,120.947576
...,...
QIRL,111.687296
ZU273983,118.179626
ZU612255,70.256126


In [7]:
#get the proteomics dataframe
pro = adi.get_snapshot('proteomics_corrected')
pro = pro.sort_values(by='days_in_program')
#drop the nas first
pro = pro.drop_duplicates('public_client_id')
pro = pro.set_index('public_client_id')
pro = pro.iloc[:,20:]
pro_zerocounts=(pro == 0).sum() # sum up times any columns have zero values
pro_nonnancounts=(~pro.isna()).sum() # sum up times any columns have non-NA values
pro = pro.loc[:, pro_nonnancounts/pro.shape[0] >= .70] #this sum of zero values/num of total rows should less than 30% to proceed with the cohort
pro = pro.fillna(pro.median())
pro

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,CVD2_O00182,CVD2_O00220,CVD2_O00253,CVD2_O14763,CVD2_O14836,CVD2_O43915,CVD2_O94907,CVD2_P00797,CVD2_P01127,CVD2_P01241,CVD2_P01730,CVD2_P01833,CVD2_P02760,CVD2_P04179,CVD2_P04792,CVD2_P05231,CVD2_P06858,CVD2_P07204,CVD2_P07585,CVD2_P07711,CVD2_P09237,CVD2_P09341,CVD2_P09601,CVD2_P09874,CVD2_P10147,CVD2_P12104,CVD2_P12931,CVD2_P13726,CVD2_P16860,CVD2_P18510,CVD2_P19883,CVD2_P21583,CVD2_P21980,CVD2_P22004,CVD2_P24394,CVD2_P25116,CVD2_P26022,CVD2_P27352,CVD2_P29965,CVD2_P31994,CVD2_P31997,CVD2_P35218,CVD2_P35318,CVD2_P35442,CVD2_P35475,CVD2_P39900,CVD2_P40225,CVD2_P41159,CVD2_P47992,CVD2_P49763,CVD2_P51161,CVD2_P51888,CVD2_P78380,CVD2_Q02763,CVD2_Q04760,CVD2_Q12866,CVD2_Q13043,CVD2_Q13219,CVD2_Q14005,CVD2_Q14116,CVD2_Q14242,CVD2_Q15109,CVD2_Q15389,CVD2_Q16651,CVD2_Q16698,CVD2_Q76LX8,CVD2_Q8IW75,CVD2_Q8IYS5,"CVD2_Q8NEV9,Q14213",CVD2_Q8TAD2,CVD2_Q92583,CVD2_Q96D42,CVD2_Q96IQ7,CVD2_Q99075,CVD2_Q99523,CVD2_Q99895,CVD2_Q9BQ51,CVD2_Q9BQR3,CVD2_Q9BUD6,CVD2_Q9BWV1,CVD2_Q9BYF1,CVD2_Q9GZV9,CVD2_Q9HB29,CVD2_Q9NQ25,CVD2_Q9NSA1,CVD2_Q9UEW3,CVD2_Q9UIB8,CVD2_Q9UJM8,CVD2_Q9UK05,CVD2_Q9UKP3,CVD2_Q9Y6K9,CVD2_Q9Y6Q6,CVD3_NT-proBNP,CVD3_O00175,CVD3_O00300,CVD3_O14798,CVD3_O15467,CVD3_O75594,CVD3_O95998,CVD3_P00533,...,CVD3_Q9H2A7,CVD3_Q9HCB6,CVD3_Q9HD89,CVD3_Q9NPY3,CVD3_Q9NQ76,CVD3_Q9UBR2,CVD3_Q9UM47,CVD3_Q9Y275,CVD3_Q9Y624,INF_O00300,INF_O14625,INF_O14788,INF_O15169,INF_O15444,INF_O43508,INF_O43557,INF_O95630,INF_O95750,INF_O95760,INF_P00749,INF_P00813,INF_P01135,INF_P01137,INF_P01138,INF_P01374,INF_P01375,INF_P01579,INF_P01583,INF_P02778,INF_P03956,INF_P05112,INF_P05113,INF_P05231,INF_P06127,INF_P09238,INF_P09341,INF_P09603,INF_P10145,INF_P10147,INF_P13232,INF_P13236,INF_P13500,INF_P13725,INF_P14210,INF_P14784,INF_P15018,INF_P15692,INF_P20783,INF_P21583,INF_P22301,INF_P25942,INF_P28325,INF_P29460,INF_P35225,INF_P39905,INF_P42702,INF_P42830,INF_P49771,INF_P50225,INF_P50591,INF_P51671,INF_P55773,INF_P60568,INF_P78423,INF_P78556,INF_P80075,INF_P80098,INF_P80162,INF_P80511,INF_Q07011,INF_Q07325,INF_Q08334,INF_Q13007,INF_Q13261,INF_Q13291,INF_Q13478,INF_Q13541,INF_Q13651,INF_Q14116,INF_Q14790,INF_Q16552,INF_Q5T4W7,INF_Q8IXJ6,INF_Q8N6P7,INF_Q8NF90,INF_Q8NFT8,INF_Q8WWJ7,INF_Q969D9,INF_Q99616,INF_Q99731,INF_Q99748,INF_Q9BZW8,INF_Q9GZV9,INF_Q9H5V8,INF_Q9NRJ3,INF_Q9NSA1,INF_Q9NYY1,INF_Q9NZQ7,INF_Q9P0M4,INF_Q9UHF4
public_client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01920535,9.443415,2.220425,3.345160,4.946235,9.042380,7.698070,8.571280,6.558815,9.791805,5.878660,5.420915,7.003370,8.232580,9.573035,11.728940,3.082755,10.394200,9.468840,5.164180,6.663880,5.563405,7.181035,11.392970,3.577085,1.869720,6.909185,8.090210,6.301665,2.594040,3.942035,10.436270,10.360375,9.284715,4.657165,2.443425,8.582250,3.222625,6.672755,5.120810,2.777915,3.689215,1.484945,8.539085,6.578855,4.915260,6.665045,3.109795,8.004395,4.004445,7.200415,1.380270,7.072135,6.977545,7.873905,5.547470,5.922365,5.918690,2.896960,7.052000,7.463325,6.011310,5.225940,9.457405,8.470255,7.608665,5.925945,-0.335835,10.199560,5.05541,2.721250,6.951315,6.432590,2.767885,4.095920,7.438985,9.167575,2.499275,8.953680,10.602865,4.843725,2.142545,2.227755,4.065410,3.528445,4.070795,7.102190,5.175995,2.497100,8.228885,3.085560,6.614850,5.575830,4.275815,2.697040,3.233495,5.180835,5.350025,6.608140,5.510990,2.668385,...,5.071630,0.652140,6.202575,10.828120,4.726670,4.443785,4.537370,6.166735,8.697655,9.233725,6.570940,4.342045,4.109595,3.631355,9.735035,3.808740,4.185050,6.014410,2.130695,9.195035,2.572830,2.601160,6.886035,1.246005,4.678345,-0.031690,0.567740,1.081505,8.193050,12.857570,-0.269985,-0.625695,2.309655,4.311935,4.683320,8.096845,9.154820,4.383710,0.331990,1.690370,0.879060,10.137015,3.037705,7.326735,1.238200,-0.692460,8.845490,0.956015,9.703860,2.699390,10.983855,7.362195,5.533530,-0.155030,1.155300,2.494180,10.149215,9.638770,3.14

In [8]:
#Get the proteomics metadata df
pro_m = adi.get_snapshot('proteomics_metadata')
pro_m = pro_m.drop_duplicates()
common_rows = set(pro.columns)
common_columns = set(pro_m['name'])
common = pd.Series(list(common_rows.intersection(common_columns)))
pro_m = pro_m[pro_m['name'].isin(common)]
pro_m.to_csv('proteomics_metadata_table.csv', index = False)
pro_m

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,name,panel,uniprot,gene_name,gene_description,gene_id,transcript_id,protein_id
185,CVD2_O00182,CVD2,O00182,LGALS9,"lectin, galactoside-binding, soluble, 9",ENSG00000168961,"ENST00000395473,ENST00000302228","ENSP00000378856,ENSP00000306228"
186,CVD2_O00220,CVD2,O00220,TNFRSF10A,"tumor necrosis factor receptor superfamily, me...",ENSG00000104689,ENST00000221132,ENSP00000221132
187,CVD2_O00253,CVD2,O00253,AGRP,agouti related protein homolog (mouse),ENSG00000159723,ENST00000290953,ENSP00000290953
188,CVD2_O14763,CVD2,O14763,TNFRSF10B,"tumor necrosis factor receptor superfamily, me...",ENSG00000120889,"ENST00000276431,ENST00000347739","ENSP00000276431,ENSP00000317859"
189,CVD2_O14836,CVD2,O14836,TNFRSF13B,"tumor necrosis factor receptor superfamily, me...",ENSG00000240505,"ENST00000261652,ENST00000583789","ENSP00000261652,ENSP00000462952"
...,...,...,...,...,...,...,...,...
548,INF_Q9NSA1,INF,Q9NSA1,FGF21,fibroblast growth factor 21,ENSG00000105550,"ENST00000593756,ENST00000222157","ENSP00000471477,ENSP00000222157"
549,INF_Q9NYY1,INF,Q9NYY1,IL20,interleukin 20,ENSG00000162891,"ENST00000367098,ENST00000367096","ENSP00000356065,ENSP00000356063"
550,INF_Q9NZQ7,INF,Q9NZQ7,CD274,CD274 molecule,ENSG00000120217,"ENST00000381577,ENST00000381573","ENSP00000370989,ENSP00000370985"
551,INF_Q9P0M4,INF,Q9P0M4,IL17C,interleukin 17C,ENSG00000124391,ENST00000244241,ENSP00000244241


In [9]:
#merge the datasets among common public_client_id
pro_fin = pro
pro_fin = pd.merge(bowel, pro_fin, left_index = True, right_index = True)
pro_fin = pd.merge(biodf, pro_fin, left_index = True, right_index = True)
pro_fin = pd.merge(bmi, pro_fin, left_index = True, right_index = True)
pro_fin = pd.merge(eGFR, pro_fin, left_index = True, right_index = True)
pro_fin = pro_fin.drop_duplicates()
pro_fin

,eGFR,BMI_CALC,sex,age,bowel,CVD2_O00182,CVD2_O00220,CVD2_O00253,CVD2_O14763,CVD2_O14836,CVD2_O43915,CVD2_O94907,CVD2_P00797,CVD2_P01127,CVD2_P01241,CVD2_P01730,CVD2_P01833,CVD2_P02760,CVD2_P04179,CVD2_P04792,CVD2_P05231,CVD2_P06858,CVD2_P07204,CVD2_P07585,CVD2_P07711,CVD2_P09237,CVD2_P09341,CVD2_P09601,CVD2_P09874,CVD2_P10147,CVD2_P12104,CVD2_P12931,CVD2_P13726,CVD2_P16860,CVD2_P18510,CVD2_P19883,CVD2_P21583,CVD2_P21980,CVD2_P22004,CVD2_P24394,CVD2_P25116,CVD2_P26022,CVD2_P27352,CVD2_P29965,CVD2_P31994,CVD2_P31997,CVD2_P35218,CVD2_P35318,CVD2_P35442,CVD2_P35475,CVD2_P39900,CVD2_P40225,CVD2_P41159,CVD2_P47992,CVD2_P49763,CVD2_P51161,CVD2_P51888,CVD2_P78380,CVD2_Q02763,CVD2_Q04760,CVD2_Q12866,CVD2_Q13043,CVD2_Q13219,CVD2_Q14005,CVD2_Q14116,CVD2_Q14242,CVD2_Q15109,CVD2_Q15389,CVD2_Q16651,CVD2_Q16698,CVD2_Q76LX8,CVD2_Q8IW75,CVD2_Q8IYS5,"CVD2_Q8NEV9,Q14213",CVD2_Q8TAD2,CVD2_Q92583,CVD2_Q96D42,CVD2_Q96IQ7,CVD2_Q99075,CVD2_Q99523,CVD2_Q99895,CVD2_Q9BQ51,CVD2_Q9BQR3,CVD2_Q9BUD6,CVD2_Q9BWV1,CVD2_Q9BYF1,CVD2_Q9GZV9,CVD2_Q9HB29,CVD2_Q9NQ25,CVD2_Q9NSA1,CVD2_Q9UEW3,CVD2_Q9UIB8,CVD2_Q9UJM8,CVD2_Q9UK05,CVD2_Q9UKP3,CVD2_Q9Y6K9,CVD2_Q9Y6Q6,CVD3_NT-proBNP,CVD3_O00175,CVD3_O00300,...,CVD3_Q9H2A7,CVD3_Q9HCB6,CVD3_Q9HD89,CVD3_Q9NPY3,CVD3_Q9NQ76,CVD3_Q9UBR2,CVD3_Q9UM47,CVD3_Q9Y275,CVD3_Q9Y624,INF_O00300,INF_O14625,INF_O14788,INF_O15169,INF_O15444,INF_O43508,INF_O43557,INF_O95630,INF_O95750,INF_O95760,INF_P00749,INF_P00813,INF_P01135,INF_P01137,INF_P01138,INF_P01374,INF_P01375,INF_P01579,INF_P01583,INF_P02778,INF_P03956,INF_P05112,INF_P05113,INF_P05231,INF_P06127,INF_P09238,INF_P09341,INF_P09603,INF_P10145,INF_P10147,INF_P13232,INF_P13236,INF_P13500,INF_P13725,INF_P14210,INF_P14784,INF_P15018,INF_P15692,INF_P20783,INF_P21583,INF_P22301,INF_P25942,INF_P28325,INF_P29460,INF_P35225,INF_P39905,INF_P42702,INF_P42830,INF_P49771,INF_P50225,INF_P50591,INF_P51671,INF_P55773,INF_P60568,INF_P78423,INF_P78556,INF_P80075,INF_P80098,INF_P80162,INF_P80511,INF_Q07011,INF_Q07325,INF_Q08334,INF_Q13007,INF_Q13261,INF_Q13291,INF_Q13478,INF_Q13541,INF_Q13651,INF_Q14116,INF_Q14790,INF_Q16552,INF_Q5T4W7,INF_Q8IXJ6,INF_Q8N6P7,INF_Q8NF90,INF_Q8NFT8,INF_Q8WWJ7,INF_Q969D9,INF_Q99616,INF_Q99731,INF_Q99748,INF_Q9BZW8,INF_Q9GZV9,INF_Q9H5V8,INF_Q9NRJ3,INF_Q9NSA1,INF_Q9NYY1,INF_Q9NZQ7,INF_Q9P0M4,INF_Q9UHF4
public_client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01001621,69.439409,23.627456,F,54.0,2,9.326155,3.344780,4.911965,5.512895,9.372900,8.742650,9.992035,7.466190,10.473635,11.165985,5.911860,7.276935,8.658955,9.751730,11.365455,3.762180,10.874030,10.481640,6.110360,7.381725,10.077430,9.241960,11.479880,3.056585,2.633770,9.760320,8.199150,7.176440,1.572715,3.912710,11.449405,11.018225,7.234650,6.462795,3.223600,10.051485,4.249090,4.935210,7.383455,3.284140,4.100805,1.707005,8.492990,6.578760,6.435470,6.992530,3.963345,6.374700,5.331580,8.616225,2.749635,7.551500,6.886045,8.568240,7.044670,7.200480,7.219360,3.537255,6.91347,8.211615,6.400250,5.938545,9.900685,9.473325,7.557365,6.124760,1.763210,11.070590,5.954700,3.483000,10.207225,7.403630,3.676795,5.258390,8.266245,11.513225,3.934500,10.023040,10.926525,5.793625,3.105820,3.849705,5.066030,3.985630,7.590035,7.721935,6.603090,2.171630,8.092525,5.542415,7.278035,6.120900,4.147015,6.422920,4.445995,...,6.048045,1.166730,6.464805,11.409885,6.561390,5.293715,5.346620,7.715980,5.814255,10.473760,6.944920,4.498970,4.943280,5.818665,10.585000,4.255930,5.514030,7.030860,1.244925,10.197305,3.396830,3.270900,7.521820,1.480165,4.708905,0.272995,0.419540,0.899865,8.004730,13.331425,-0.586585,-0.010395,2.796655,4.481675,5.763415,9.808175,9.801320,5.198730,0.904825,3.436085,1.765880,10.799995,3.932575,8.324480,1.045350,-0.790945,10.322615,1.233610,10.272160,3.212775,10.889035,7.244350,5.271450,-0.442855,2.542635,3.488550,12.141750,10.313720,3.593495,8.496275,8.178315,10.257795,0.5

In [10]:
#Grabbing Metadata:
pro_fin.iloc[:,:5].to_csv('proteomics_metadata.csv')

#count data
pro_fin.iloc[:,5:].transpose().to_csv('proteomics.csv',)

#full data
pro_fin.to_csv('proteomics_full.csv')

In [11]:
pro_fin

,eGFR,BMI_CALC,sex,age,bowel,CVD2_O00182,CVD2_O00220,CVD2_O00253,CVD2_O14763,CVD2_O14836,CVD2_O43915,CVD2_O94907,CVD2_P00797,CVD2_P01127,CVD2_P01241,CVD2_P01730,CVD2_P01833,CVD2_P02760,CVD2_P04179,CVD2_P04792,CVD2_P05231,CVD2_P06858,CVD2_P07204,CVD2_P07585,CVD2_P07711,CVD2_P09237,CVD2_P09341,CVD2_P09601,CVD2_P09874,CVD2_P10147,CVD2_P12104,CVD2_P12931,CVD2_P13726,CVD2_P16860,CVD2_P18510,CVD2_P19883,CVD2_P21583,CVD2_P21980,CVD2_P22004,CVD2_P24394,CVD2_P25116,CVD2_P26022,CVD2_P27352,CVD2_P29965,CVD2_P31994,CVD2_P31997,CVD2_P35218,CVD2_P35318,CVD2_P35442,CVD2_P35475,CVD2_P39900,CVD2_P40225,CVD2_P41159,CVD2_P47992,CVD2_P49763,CVD2_P51161,CVD2_P51888,CVD2_P78380,CVD2_Q02763,CVD2_Q04760,CVD2_Q12866,CVD2_Q13043,CVD2_Q13219,CVD2_Q14005,CVD2_Q14116,CVD2_Q14242,CVD2_Q15109,CVD2_Q15389,CVD2_Q16651,CVD2_Q16698,CVD2_Q76LX8,CVD2_Q8IW75,CVD2_Q8IYS5,"CVD2_Q8NEV9,Q14213",CVD2_Q8TAD2,CVD2_Q92583,CVD2_Q96D42,CVD2_Q96IQ7,CVD2_Q99075,CVD2_Q99523,CVD2_Q99895,CVD2_Q9BQ51,CVD2_Q9BQR3,CVD2_Q9BUD6,CVD2_Q9BWV1,CVD2_Q9BYF1,CVD2_Q9GZV9,CVD2_Q9HB29,CVD2_Q9NQ25,CVD2_Q9NSA1,CVD2_Q9UEW3,CVD2_Q9UIB8,CVD2_Q9UJM8,CVD2_Q9UK05,CVD2_Q9UKP3,CVD2_Q9Y6K9,CVD2_Q9Y6Q6,CVD3_NT-proBNP,CVD3_O00175,CVD3_O00300,...,CVD3_Q9H2A7,CVD3_Q9HCB6,CVD3_Q9HD89,CVD3_Q9NPY3,CVD3_Q9NQ76,CVD3_Q9UBR2,CVD3_Q9UM47,CVD3_Q9Y275,CVD3_Q9Y624,INF_O00300,INF_O14625,INF_O14788,INF_O15169,INF_O15444,INF_O43508,INF_O43557,INF_O95630,INF_O95750,INF_O95760,INF_P00749,INF_P00813,INF_P01135,INF_P01137,INF_P01138,INF_P01374,INF_P01375,INF_P01579,INF_P01583,INF_P02778,INF_P03956,INF_P05112,INF_P05113,INF_P05231,INF_P06127,INF_P09238,INF_P09341,INF_P09603,INF_P10145,INF_P10147,INF_P13232,INF_P13236,INF_P13500,INF_P13725,INF_P14210,INF_P14784,INF_P15018,INF_P15692,INF_P20783,INF_P21583,INF_P22301,INF_P25942,INF_P28325,INF_P29460,INF_P35225,INF_P39905,INF_P42702,INF_P42830,INF_P49771,INF_P50225,INF_P50591,INF_P51671,INF_P55773,INF_P60568,INF_P78423,INF_P78556,INF_P80075,INF_P80098,INF_P80162,INF_P80511,INF_Q07011,INF_Q07325,INF_Q08334,INF_Q13007,INF_Q13261,INF_Q13291,INF_Q13478,INF_Q13541,INF_Q13651,INF_Q14116,INF_Q14790,INF_Q16552,INF_Q5T4W7,INF_Q8IXJ6,INF_Q8N6P7,INF_Q8NF90,INF_Q8NFT8,INF_Q8WWJ7,INF_Q969D9,INF_Q99616,INF_Q99731,INF_Q99748,INF_Q9BZW8,INF_Q9GZV9,INF_Q9H5V8,INF_Q9NRJ3,INF_Q9NSA1,INF_Q9NYY1,INF_Q9NZQ7,INF_Q9P0M4,INF_Q9UHF4
public_client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01001621,69.439409,23.627456,F,54.0,2,9.326155,3.344780,4.911965,5.512895,9.372900,8.742650,9.992035,7.466190,10.473635,11.165985,5.911860,7.276935,8.658955,9.751730,11.365455,3.762180,10.874030,10.481640,6.110360,7.381725,10.077430,9.241960,11.479880,3.056585,2.633770,9.760320,8.199150,7.176440,1.572715,3.912710,11.449405,11.018225,7.234650,6.462795,3.223600,10.051485,4.249090,4.935210,7.383455,3.284140,4.100805,1.707005,8.492990,6.578760,6.435470,6.992530,3.963345,6.374700,5.331580,8.616225,2.749635,7.551500,6.886045,8.568240,7.044670,7.200480,7.219360,3.537255,6.91347,8.211615,6.400250,5.938545,9.900685,9.473325,7.557365,6.124760,1.763210,11.070590,5.954700,3.483000,10.207225,7.403630,3.676795,5.258390,8.266245,11.513225,3.934500,10.023040,10.926525,5.793625,3.105820,3.849705,5.066030,3.985630,7.590035,7.721935,6.603090,2.171630,8.092525,5.542415,7.278035,6.120900,4.147015,6.422920,4.445995,...,6.048045,1.166730,6.464805,11.409885,6.561390,5.293715,5.346620,7.715980,5.814255,10.473760,6.944920,4.498970,4.943280,5.818665,10.585000,4.255930,5.514030,7.030860,1.244925,10.197305,3.396830,3.270900,7.521820,1.480165,4.708905,0.272995,0.419540,0.899865,8.004730,13.331425,-0.586585,-0.010395,2.796655,4.481675,5.763415,9.808175,9.801320,5.198730,0.904825,3.436085,1.765880,10.799995,3.932575,8.324480,1.045350,-0.790945,10.322615,1.233610,10.272160,3.212775,10.889035,7.244350,5.271450,-0.442855,2.542635,3.488550,12.141750,10.313720,3.593495,8.496275,8.178315,10.257795,0.5